In [4]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from config import password

### Store CSV --Top 50 School Districts-- into DataFrame

In [5]:
csv_file = "../Transform/top50_school_district_clean.csv"
district_df = pd.read_csv(csv_file).iloc[:,1:]

district_df.head()

,name,rating,city,niche_grade,number_school,number_student
0,Adlai E. Stevenson High School District No. 125,#1 Best School Districts in Chicago Area,"LINCOLNSHIRE, IL",grade A+,2,4271
1,Community High School District 128,#2 Best School Districts in Chicago Area,"VERNON HILLS, IL",grade A+,2,3287
2,New Trier Township High School District No. 203,#3 Best School Districts in Chicago Area,"NORTHFIELD, IL",grade A+,2,4040
3,Glenbrook High Schools District 225,#4 Best School Districts in Chicago Area,"GLENVIEW, IL",grade A+,4,5201
4,Township High School District No. 113,#5 Best School Districts in Chicago Area,"HIGHLAND PARK, IL",grade A+,2,3467


### Store CSV --Top 500 Elementery Schools-- into DataFrame

In [6]:
csv_file = "../Transform/top500_elementary_school_clean.csv"
school_df = pd.read_csv(csv_file).iloc[:,1:]

school_df.head()


,name,rating,district,niche_grade,stu_number,ratio
0,Meadow Glens Elementary School,#1 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,494,15:1
1,Barbara B. Rose Elementary School,#2 Best Public Elementary Schools in Chicago Area,Barrington Community Unit School District No. 220,grade A+,436,13:1
2,Grove Avenue Elementary School,#3 Best Public Elementary Schools in Chicago Area,Barrington Community Unit School District No. 220,grade A+,520,15:1
3,Ellsworth Elementary School,#4 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,258,14:1
4,Steeple Run Elementary School,#5 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,629,16:1


### Connect to local Postgres database

In [7]:
rds_connection_string = f"postgres:{password}@localhost:5432/CMAPSA"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [8]:
engine.table_names()

<ipython-input-8-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['district', 'school']

### Use pandas to load DataFrames into Postgres database tables

In [9]:
## Truncate the table
engine.execute("TRUNCATE TABLE district")
 

In [10]:
## Read Empty Table
pd.read_sql_query('select * from district', con=engine).head(100)

,id,name,rating,city,niche_grade,number_school,number_student


In [11]:
## Load DataFrame into Table
district_df.to_sql(name='district', con=engine, if_exists='append', index=False)

In [12]:
##Confirm data has been added by querying the table
pd.read_sql_query('select * from district', con=engine).head(100)

,id,name,rating,city,niche_grade,number_school,number_student
0,751,Adlai E. Stevenson High School District No. 125,#1 Best School Districts in Chicago Area,"LINCOLNSHIRE, IL",grade A+,2,4271
1,752,Community High School District 128,#2 Best School Districts in Chicago Area,"VERNON HILLS, IL",grade A+,2,3287
2,753,New Trier Township High School District No. 203,#3 Best School Districts in Chicago Area,"NORTHFIELD, IL",grade A+,2,4040
3,754,Glenbrook High Schools District 225,#4 Best School Districts in Chicago Area,"GLENVIEW, IL",grade A+,4,5201
4,755,Township High School District No. 113,#5 Best School Districts in Chicago Area,"HIGHLAND PARK, IL",grade A+,2,3467
5,756,Hinsdale Township High School District No. 86,#6 Best School Districts in Chicago Area,"HINSDALE, IL",grade A+,3,4146
6,757,Naperville Community Unit School District No. 203,#7 Best School Districts in Chicago Area,"NAPERVILLE, IL",grade A+,22,16586
7,758,Niles Township Community High School District ...,#8 Best School Districts in Chicago Area,"SKOKIE, IL",grade A+,3,4592
8,759,Barrington Community Unit School District No. 220,#9 Best School Districts in Chicago Area,"BARRINGTON, IL",grade A+,12,8557
9,760,Township High School District No. 211,#10 Best School Districts in Chicago Area,"PALATINE, IL",grade A+,7,11855


In [13]:
## Truncate the table
engine.execute("TRUNCATE TABLE school")

In [14]:
## Read Empty Table
pd.read_sql_query('select * from school', con=engine).head(100)

,id,name,rating,district,niche_grade,stu_number,ratio


In [15]:
## Load DataFrame into Table
school_df.to_sql(name='school', con=engine, if_exists='append', index=False)

In [16]:
##Confirm data has been added by querying the table
pd.read_sql_query('select * from school', con=engine).head(100)

,id,name,rating,district,niche_grade,stu_number,ratio
0,6945,Meadow Glens Elementary School,#1 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,494,15:1
1,6946,Barbara B. Rose Elementary School,#2 Best Public Elementary Schools in Chicago Area,Barrington Community Unit School District No. 220,grade A+,436,13:1
2,6947,Grove Avenue Elementary School,#3 Best Public Elementary Schools in Chicago Area,Barrington Community Unit School District No. 220,grade A+,520,15:1
3,6948,Ellsworth Elementary School,#4 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,258,14:1
4,6949,Steeple Run Elementary School,#5 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,629,16:1
...,...,...,...,...,...,...,...
95,7040,Bell-Graham Elementary School,#96 Best Public Elementary Schools in Chicago ...,St. Charles Community Unit School District No....,grade A,440,15:1
96,7041,Romona Elementary School,#97 Best Public Elementary Schools in Chicago ...,Wilmette School District No. 39,grade A,532,12:1
97,7042,Bower Elementary School,#98 Best Public Elementary Schools in Chicago ...,Community Unit School District No. 200,grade A,441,12:1
98,7043,Jackson Elementary School,#99 Best Public Elementary Schools in Chicago ...,Elmhurst School District No. 205,grade A,457,14:1


### Join Top Elementery Schools and Top School Districts

In [17]:
school_df['district']=school_df['district'].str.strip()
district_df['name']=district_df['name'].str.strip()

In [18]:
school_df['district'].head(10)

0    Naperville Community Unit School District No. 203
1    Barrington Community Unit School District No. 220
2    Barrington Community Unit School District No. 220
3    Naperville Community Unit School District No. 203
4    Naperville Community Unit School District No. 203
5    Naperville Community Unit School District No. 203
6    Naperville Community Unit School District No. 203
7    Naperville Community Unit School District No. 203
8    Indian Prairie Community Unit School District ...
9    Naperville Community Unit School District No. 203
Name: district, dtype: object

In [19]:
district_df['name'].head(10)

0      Adlai E. Stevenson High School District No. 125
1                   Community High School District 128
2      New Trier Township High School District No. 203
3                  Glenbrook High Schools District 225
4                Township High School District No. 113
5        Hinsdale Township High School District No. 86
6    Naperville Community Unit School District No. 203
7    Niles Township Community High School District ...
8    Barrington Community Unit School District No. 220
9                Township High School District No. 211
Name: name, dtype: object

In [20]:
## Associate a School District with each school :Inner Join
school_district_lj_df = pd.merge(school_df, district_df, how='left', left_on = 'district', right_on = 'name')

In [23]:
## Display Joined DataSets
school_district_lj_df.head()

,name_x,rating_x,district,niche_grade_x,stu_number,ratio,name_y,rating_y,city,niche_grade_y,number_school,number_student
0,Meadow Glens Elementary School,#1 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,494,15:1,Naperville Community Unit School District No. 203,#7 Best School Districts in Chicago Area,"NAPERVILLE, IL",grade A+,22.0,16586.0
1,Barbara B. Rose Elementary School,#2 Best Public Elementary Schools in Chicago Area,Barrington Community Unit School District No. 220,grade A+,436,13:1,Barrington Community Unit School District No. 220,#9 Best School Districts in Chicago Area,"BARRINGTON, IL",grade A+,12.0,8557.0
2,Grove Avenue Elementary School,#3 Best Public Elementary Schools in Chicago Area,Barrington Community Unit School District No. 220,grade A+,520,15:1,Barrington Community Unit School District No. 220,#9 Best School Districts in Chicago Area,"BARRINGTON, IL",grade A+,12.0,8557.0
3,Ellsworth Elementary School,#4 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,258,14:1,Naperville Community Unit School District No. 203,#7 Best School Districts in Chicago Area,"NAPERVILLE, IL",grade A+,22.0,16586.0
4,Steeple Run Elementary School,#5 Best Public Elementary Schools in Chicago Area,Naperville Community Unit School District No. 203,grade A+,629,16:1,Naperville Community Unit School District No. 203,#7 Best School Districts in Chicago Area,"NAPERVILLE, IL",grade A+,22.0,16586.0


In [24]:
school_district_lj_df.to_csv('./top500_elementary_school_withdistrict_clean.csv')